In [4]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from unc.utils import load_info
from definitions import ROOT_DIR

In [5]:
results_fname = Path(ROOT_DIR, "results", "lobster_data.npy")
loaded = load_info(results_fname)

In [7]:
obs_res = loaded['2']
unc_res = loaded['2o']

In [18]:
traj = obs_res['obs'][0]

def get_distilled_obs(traj):
    # first we get position
    pos_one_hot = traj[:, :3]
    pos_traj = np.nonzero(pos_one_hot)[-1]
    ot1 = traj[:, 3]
    ot2 = traj[:, 6]
    reduced_obses = np.stack([pos_traj, ot1, ot2], axis=-1)
    return reduced_obses

In [17]:
reduced_unc_obses = 

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [2., 0., 0.],
       [2., 0., 1.],
       [2., 0., 1.],
       [0., 0., 0.],
       [2., 0., 1.],
       [2., 0., 1.],
       [2., 0., 1.],
       [2., 0., 1.],
       [2., 0., 1.],
       [0., 0., 0.],
       [2., 0., 1.],
       [2., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [2., 0., 1.],
       [2., 0., 1.],
       [2., 0., 1.],
       [2., 0., 1.],
       [2., 0., 1.],
       [0., 0., 0.],
       [2., 0., 1.],
       [2., 0., 1.],
       [0., 0., 0.],
       [2., 0., 1.],
       [2., 0., 1.],
       [2., 0., 0.],
       [2., 0., 1.],
       [2., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [2., 0., 1.],
       [2., 0., 1.],
       [2., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 0., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1